# Compute performance metrics for the given Y and Y_score without sklearn

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'none'

In [2]:
import numpy as np
import pandas as pd

In [3]:
print (pd.options.display.max_columns)

20


<pre>
<font color='red'><b>A.</b></font> Compute performance metrics for the given data <strong>5_a.csv</strong>
   <b>Note 1:</b> in this data you can see number of positive points >> number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_a.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score, you need to compute different thresholds and for each threshold compute tpr,fpr and then use               numpy.trapz(tpr_array, fpr_array) <a href='https://stackoverflow.com/q/53603376/4084039'>https://stackoverflow.com/q/53603376/4084039</a>, <a href='https://stackoverflow.com/a/39678975/4084039'>https://stackoverflow.com/a/39678975/4084039</a> Note: it should be numpy.trapz(tpr_array, fpr_array) not numpy.trapz(fpr_array, tpr_array)</li>
<li> Compute Accuracy Score </li>
</ol>
</pre>

In [4]:
!ls

5_a.csv
5_b.csv
5_c.csv
5_d.csv
5_Performance_metrics_Instructions.ipynb


In [5]:
df = pd.read_csv('5_a.csv')
print(df.shape)
print(df.head())

(10100, 2)
     y     proba
0  1.0  0.637387
1  1.0  0.635165
2  1.0  0.766586
3  1.0  0.724564
4  1.0  0.889199


In [6]:
"""
Given DataFrame column `y' contains float value, here we convert it to int
"""
def convert_to_int(df, col):
    
    df[col] = df[col].map(lambda y: int(y))
    
    return

In [7]:
convert_to_int(df, 'y')

df_top_k_rows = df.head(n = 3)

print(df_top_k_rows)

   y     proba
0  1  0.637387
1  1  0.635165
2  1  0.766586


> 1.1 Derive the class labels from given score

In [8]:
def derive_class_label(df, ref_col_name, new_col_name, thld):
    
    df[new_col_name] = df[ref_col_name].map(lambda prob: 1 if prob > thld else 0)
    
    return

In [9]:
derive_class_label(df, 'proba', 'y_hat', 0.5)
df_top_k_rows = df.head(n = 3)

print(df_top_k_rows)

   y     proba  y_hat
0  1  0.637387      1
1  1  0.635165      1
2  1  0.766586      1


> 1.2 Compute Confusion Matrix

In [10]:
InteractiveShell.ast_node_interactivity = 'none'

In [11]:
"""
confusion_mtx_ref: 
    key: tuple(original output, predicted output)
    
    value: dict({})
        name: element notation ( 'tp', 'fp', 'fn', 'tn')
        mtx_idx: Contains matrix indices, to get confusion matx element as per notation
"""

confusion_mtx_ref = {
    (1, 1): {'name': 'tp', 'mtx_idx': (0, 0)},
    (0, 1): {'name': 'fp', 'mtx_idx': (0, 1)},
    (1, 0): {'name': 'fn', 'mtx_idx': (1, 0)},
    (0, 0): {'name': 'tn', 'mtx_idx': (1, 1)}
}

print(confusion_mtx_ref)

{(1, 1): {'name': 'tp', 'mtx_idx': (0, 0)}, (0, 1): {'name': 'fp', 'mtx_idx': (0, 1)}, (1, 0): {'name': 'fn', 'mtx_idx': (1, 0)}, (0, 0): {'name': 'tn', 'mtx_idx': (1, 1)}}


In [12]:
def get_confusion_mtx_element_ref(confusion_mtx_ref):
    
    cf_mtx_elem_ref = dict()
    for (row, col) in confusion_mtx_ref:

        temp = confusion_mtx_ref.get((row, col))

        """
        """
        conf_mtx_elem_not = temp.get('name')

        """
        """
        conf_mtx_elem_index = temp.get('mtx_idx')

        cf_mtx_elem_ref.update({conf_mtx_elem_not: conf_mtx_elem_index})

    return cf_mtx_elem_ref

cf_mtx_elem_ref = get_confusion_mtx_element_ref(confusion_mtx_ref)
print(cf_mtx_elem_ref)

{'tp': (0, 0), 'fp': (0, 1), 'fn': (1, 0), 'tn': (1, 1)}


In [13]:
cf_mtx_elem_to_idx_mapping = {elem_nota_numeric: confusion_mtx_ref.get(elem_nota_numeric).get('mtx_idx') for elem_nota_numeric in confusion_mtx_ref}
print(cf_mtx_elem_to_idx_mapping)

{(1, 1): (0, 0), (0, 1): (0, 1), (1, 0): (1, 0), (0, 0): (1, 1)}


In [14]:
def get_confusion_mtx(series_i, series_ii, cf_mtx_elem_to_idx_mapping):
    
    confusion_mtx = np.zeros((2, 2), dtype = int)
    
    for index, item_y in enumerate(series_i):

        item_y_hat = series_ii.get(index)

        x, y = cf_mtx_elem_to_idx_mapping.get((item_y, item_y_hat))

        cf_item_count = confusion_mtx[x, y]

        confusion_mtx[x, y] = cf_item_count + 1

    return confusion_mtx

In [15]:
confusion_mtx = get_confusion_mtx(df['y'], df['y_hat'], cf_mtx_elem_to_idx_mapping)
print(confusion_mtx)

[[10000   100]
 [    0     0]]


> 1.3 Compute F1 Score

In [16]:
def compute_confusion_mtx_elements(confusion_mtx, cf_mtx_elem):
    """Compute confusin matrix elements i.e 
        {'tp', 'fp', 'fn', 'tn'}

    Parameters
    ----------
    confusion_mtx: numpy 2-D array
    cf_mtx_elem: dict()

    Return
    ------
    cf_mtx_elem_values: dict
        Contains mapping of confusion matrix element notation to their value (in confusion matrix itself)
        Structure: {
        'tp': 4, 'fp': 8, 'fn':5, 'tn': 7
        }
    """

    cf_mtx_elem_values = dict()
    for cf_mtx_elem_notation in cf_mtx_elem:
        """
        cf_mtx_elem_notation: string
            Confusion matrix element notation 
            Exp: {'tp', 'fp', 'fn', 'tn'}
        """

        x, y = cf_mtx_elem.get(cf_mtx_elem_notation)
        val = confusion_mtx[x, y]

        cf_mtx_elem_values.update({cf_mtx_elem_notation: val})


    return cf_mtx_elem_values

In [17]:
"""
precision = tp / (tp + fp)

recall = tp / (tp + fn)

F1 score = 2 * precision * recall / (precision + recall)
"""
def compute_tpr_fpr(cf_mtx_elem_values):
    
    tp = cf_mtx_elem_values.get('tp')
    fp = cf_mtx_elem_values.get('fp')
    fn = cf_mtx_elem_values.get('fn')
    tn = cf_mtx_elem_values.get('tn')
    
    tpr = tp / (tp + fn)
    
    fpr = fp / (fp + tn)
    
    return tpr, fpr

In [18]:
cf_mtx_elem_values = compute_confusion_mtx_elements(confusion_mtx, cf_mtx_elem_ref)

In [19]:
#def compute_precision_recall_f1(cf_mtx_elem_values):
def compute_auc_terms(cf_mtx_elem_values, auc_terms):
    
    auc_terms_ret = dict()
    auc_term_dict = dict()
    
    tp = cf_mtx_elem_values.get('tp')
    fp = cf_mtx_elem_values.get('fp')
    fn = cf_mtx_elem_values.get('fn')
    tn = cf_mtx_elem_values.get('tn')
    
    precision = tp / (tp + fp)
    
    recall = tp / (tp + fn)
    
    f1_score = 2 * precision * recall / (precision + recall)
    
    tpr = tp / (tp + fn)
    
    fpr = fp / (fp + tn)
    
    auc_term_dict.update({
        'tp': tp,
        'fp': fp,
        'fn': fn,
        'tn': tn,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'tpr': tpr,
        'fpr': fpr
    })
    
    auc_terms_ret = {term: auc_term_dict.get(term) for term in auc_terms}
    
    return auc_terms_ret


auc_term_obj = compute_auc_terms(cf_mtx_elem_values, ['precision', 'recall', 'f1_score'])
print(auc_term_obj['precision'])
print(auc_term_obj['recall'])
print(auc_term_obj['f1_score'])

0.9900990099009901
1.0
0.9950248756218906


In [20]:
"""

auc_terms = compute_auc_terms(cf_mtx_elem_values)

print(auc_terms)
"""

> 1.3 AUC score

In [21]:
def get_unique_values(df, col_name, order = 'DESC'):
    
    unique_proba = df[col_name].unique()
    unique_proba.sort()
    
    if order == 'DESC':
        unique_proba = unique_proba[::-1]
        
    return unique_proba

In [22]:
unique_proba = get_unique_values(df, 'proba')

unique_proba_shape = unique_proba.shape

print(type(unique_proba))
print(unique_proba_shape)
print(unique_proba[:5])

<class 'numpy.ndarray'>
(10100,)
[0.89996535 0.89982831 0.89982485 0.89981181 0.89976788]


In [23]:
pd.set_option('display.max_columns', unique_proba_shape[0] + 3)

In [24]:
def compute_auc_prop(df, output_prob_col, unique_proba, auc_props_list):

    auc_props = dict()
    tpr, fpr = list(), list()
    for index, thld in enumerate(unique_proba):

        col_name, confusion_mtx_ii = '', ''
        cf_mtx_elem_values_ii, auc_term_obj = dict(), dict()

        col_name = "thld_{}".format(index)
        df[(col_name, thld)] = df[output_prob_col].map(lambda proba_val: 1 if proba_val > thld else 0)

        confusion_mtx_ii = get_confusion_mtx(df['y'], df[(col_name, thld)], cf_mtx_elem_to_idx_mapping)

        cf_mtx_elem_values_ii = compute_confusion_mtx_elements(confusion_mtx_ii, cf_mtx_elem_ref)

        auc_term_obj = compute_auc_terms(cf_mtx_elem_values_ii, auc_props_list)

        auc_props.update({(col_name, thld): auc_term_obj})

    return auc_props

In [25]:
def get_auc_curve_lists(auc_props, auc_props_list):
    
    prop_i_list, prop_ii_list = list(), list()
    for col_name in auc_props:
        
        prop_i = auc_props.get(col_name).get(auc_props_list[0])
        prop_ii = auc_props.get(col_name).get(auc_props_list[1])
        
        
        prop_i_list.append(prop_i)
        prop_ii_list.append(prop_ii)
        
    return prop_i_list, prop_ii_list

In [26]:
#auc_props = compute_auc_prop(df, 'proba', unique_proba, ['tpr', 'fpr'])

In [27]:
"""
tpr, fpr = get_auc_curve_lists(auc_props, ['tpr', 'fpr'])
print(tpr)
print(fpr)
"""

In [28]:
"""
auc_score = np.trapz(tpr, fpr)
print(auc_score)
"""

<pre>
<font color='red'><b>B.</b></font> Compute performance metrics for the given data <strong>5_b.csv</strong>
   <b>Note 1:</b> in this data you can see number of positive points << number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_b.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score, you need to compute different thresholds and for each threshold compute tpr,fpr and then use               numpy.trapz(tpr_array, fpr_array) <a href='https://stackoverflow.com/q/53603376/4084039'>https://stackoverflow.com/q/53603376/4084039</a>, <a href='https://stackoverflow.com/a/39678975/4084039'>https://stackoverflow.com/a/39678975/4084039</a></li>
<li> Compute Accuracy Score </li>
</ol>
</pre>

<font color='red'><b>C.</b></font> Compute the best threshold (similarly to ROC curve computation) of probability which gives lowest values of metric <b>A</b> for the given data <strong>5_c.csv</strong>
<br>

you will be predicting label of a data points like this: $y^{pred}= \text{[0 if y_score < threshold  else 1]}$

$ A = 500 \times \text{number of false negative} + 100 \times \text{numebr of false positive}$

<pre>
   <b>Note 1:</b> in this data you can see number of negative points > number of positive points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_c.csv</b>
</pre>

In [29]:
# write your code
df_iii = pd.read_csv("5_c.csv")

print(df_iii.shape)
print(df_iii.head())

(2852, 2)
   y      prob
0  0  0.458521
1  0  0.505037
2  0  0.418652
3  0  0.412057
4  0  0.375579


In [30]:
convert_to_int(df_iii, 'y')

In [31]:
unique_proba_iii = get_unique_values(df_iii, 'prob')

print(unique_proba_iii.shape)
print(unique_proba_iii[:5])

(2791,)
[0.9577468  0.95143692 0.94863779 0.94409361 0.94111318]


In [32]:
pd.set_option('display.max_columns', df_iii.shape[0] + 3)

In [33]:
auc_props_iii = compute_auc_prop(df_iii, 'prob', unique_proba_iii, ['fn', 'fp'])

In [34]:
print(df_iii.shape)

(2852, 2793)


In [35]:
def get_mtx_scores(auc_props):
    
    mtx_score = dict()
    for thld_col in auc_props:

        fn = auc_props.get(thld_col).get('fn')
        fp = auc_props.get(thld_col).get('fp')
        
        score = 500 * fn + 100 * fp
        
        mtx_score.update({thld_col: score})
    
    return mtx_score

In [36]:
mtx_score = get_mtx_scores(auc_props_iii)

In [37]:
key_min = min(mtx_score.keys(), key=(lambda k: mtx_score[k]))
print(mtx_score[key_min])

141000


<pre>
<font color='red'><b>D.</b></font> Compute performance metrics(for regression) for the given data <strong>5_d.csv</strong>
    <b>Note 2:</b> use pandas or numpy to read the data from <b>5_d.csv</b>
    <b>Note 1:</b> <b>5_d.csv</b> will having two columns Y and predicted_Y both are real valued features
<ol>
<li> Compute Mean Square Error </li>
<li> Compute MAPE: https://www.youtube.com/watch?v=ly6ztgIkUxk</li>
<li> Compute R^2 error: https://en.wikipedia.org/wiki/Coefficient_of_determination#Definitions </li>
</ol>
</pre>

In [24]:
df_iv = pd.read_csv("5_d.csv")
print(df_iv.head())

       y   pred
0  101.0  100.0
1  120.0  100.0
2  131.0  113.0
3  164.0  125.0
4  154.0  152.0


> 4.1 Mean Square Error

In [25]:
mse = 0
for output, pred_output in zip(df_iv['y'], df_iv['pred']):

    mse += np.power((output - pred_output), 2)
    
mse = np.divide(mse, len(df_iv['y']))
print(mse)

177.16569974554707


> 4.3 Compute R^2 error

In [26]:
def get_series_statistics(series):
    
    return series.describe()    

In [27]:
series_y_stats = get_series_statistics(df_iv['y'])
print(series_y_stats)

count    157200.000000
mean         66.562087
std          63.714792
min           0.000000
25%          11.000000
50%          52.000000
75%         104.000000
max         440.000000
Name: y, dtype: float64


In [28]:
y_mean = series_y_bar_stats['mean']

In [29]:
df_iv['diff_y___y_mean'] = df_iv['y'].map(lambda y: y - y_mean)
df_iv['diff_pred_mean___y_mean'] = df_iv['pred'].map(lambda y_pred: y_pred - y_mean)

In [30]:
df_iv['square_diff_y__y_mean'] = df_iv['diff_y___y_mean'].map(lambda val: np.power(val, 2))


In [31]:
df_iv['square_diff_pred_mean___y_mean'] = df_iv['diff_pred_mean___y_mean'].map(lambda val: np.power(val, 2))

In [32]:
df_iv.head(n = 30)

,y,pred,diff_y___y_mean,diff_pred_mean___y_mean,square_diff_y__y_mean,square_diff_pred_mean___y_mean
0,101.0,100.0,34.516285,33.516285,1191.373929,1123.341359
1,120.0,100.0,53.516285,33.516285,2863.992759,1123.341359
2,131.0,113.0,64.516285,46.516285,4162.351029,2163.764769
3,164.0,125.0,97.516285,58.516285,9509.425838,3424.155609
4,154.0,152.0,87.516285,85.516285,7659.100138,7313.034998
5,133.0,153.0,66.516285,86.516285,4424.416169,7485.067568
6,148.0,139.0,81.516285,72.516285,6644.904718,5258.611588
7,172.0,145.0,105.516285,78.516285,11133.686398,6164.807008
8,153.0,162.0,86.516285,95.516285,7485.067568,9123.360698
9,162.0,154.0,95.516285,87.516285,9123.360698,7659.100138


In [33]:
sum_i = df_iv['square_diff_y__y_mean'].sum()
sum_ii = df_iv['square_diff_pred_mean___y_mean'].sum()

In [35]:
r_2 = np.divide(sum_ii, sum_i)
print(r_2)

0.9573353892620483
